# Text Encoder

> Wrappers for text models (e.g., BERT variants) using `transformers`, integrated with the custom tokenizer.

In [ ]:
#| default_exp model.text

In [ ]:
#| export
from pathlib import Path

try:
    import indic_clip.core
    print("Reloaded indic_clip.core")
except ModuleNotFoundError:
    print("indic_clip.core not found initially.")
    # Attempt to set sys.path if running in Colab and project cloned
    import sys
    if 'google.colab' in sys.modules:
        project_parent = '/content' # Assuming cloned into /content/indic-clip
        if Path('/content/drive/MyDrive/Indic-Clip').exists():
             project_parent = '/content/drive/MyDrive/Indic-Clip'
        if project_parent not in sys.path:
             sys.path.insert(0, project_parent)
             print(f"Added {project_parent} to sys.path")
        try:
            import indic_clip.core
            print("Imported indic_clip.core after path adjustment.")
        except ModuleNotFoundError:
            print("ERROR: Still cannot find indic_clip.core. Ensure project structure is correct.")
            print("Expected: /content/Indic-Clip/indic_clip/core.py or similar in Drive")
            # raise # Stop execution if core components missing
    else:
        project_parent = '/workspace'
        if Path('/workspace/indic-clip').exists():
             project_parent = '/workspace/indic-clip'
        if project_parent not in sys.path:
             sys.path.insert(0, project_parent)
             print(f"Added {project_parent} to sys.path")
        try:
            import indic_clip.core
            print("Imported indic_clip.core after path adjustment.")
        except ModuleNotFoundError:
            print("ERROR: Still cannot find indic_clip.core. Ensure project structure is correct.")
            print("Expected: /workspace/indic-clip/indic-clip/core.py or similar in Drive")

indic_clip.core not found initially.
Added /workspace/indic-clip to sys.path
Imported indic_clip.core after path adjustment.


In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import torch
import torch.nn as nn
from transformers import AutoModel, AutoConfig, logging as hf_logging
import logging

from fastai.text.all import *

try:
    # Import necessary components from our project
    from indic_clip.core import get_logger, setup_logging, PRETRAINED_TOKENIZER_NAME, DEFAULT_EMBED_DIM
    from indic_clip.data.tokenization import IndicBERTTokenizer # Assuming this is the wrapper class name
except ModuleNotFoundError:
    # Fallback if core not found (e.g. testing)
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger(__name__)
    def get_logger(name): return logging.getLogger(name)
    def setup_logging(): pass
    PRETRAINED_TOKENIZER_NAME = "ai4bharat/indic-bert"
    DEFAULT_EMBED_DIM = 768
    # Define a dummy tokenizer if the real one isn't available for basic testing
    class IndicBERTTokenizer:
        def __init__(self, *args, **kwargs): self.vocab_size = 30000 # Dummy value
        def tokenize(self, texts):
            # Dummy tokenization
            ids = torch.randint(0, self.vocab_size, (len(texts), 10))
            mask = torch.ones_like(ids)
            return {'input_ids': ids, 'attention_mask': mask}

setup_logging()
logger = get_logger(__name__)

# Reduce verbosity from Hugging Face's transformers library
hf_logging.set_verbosity_error()

Reloaded indic_clip.core


## Text Encoder Wrapper

In [ ]:
#| export
class TextEncoder(Module):
    """A wrapper for Hugging Face transformer models to extract text features.

    Provides a consistent interface for different text backbones (e.g., BERT variants).
    Loads pre-trained weights from Hugging Face Hub.
    Handles resizing token embeddings if the associated tokenizer has added special tokens.
    """
    def __init__(self,
                 model_name: str = PRETRAINED_TOKENIZER_NAME, # Often same as tokenizer
                 pretrained: bool = True,
                 output_dim: int | None = None,
                 tokenizer: IndicBERTTokenizer = None):
        """
        Initializes the Text Encoder.

        Args:
            model_name (str): The name or path of the Hugging Face model (e.g., 'ai4bharat/indic-bert').
            pretrained (bool): Whether to load pre-trained weights from Hugging Face Hub.
            output_dim (int | None): Optional dimension for a final projection layer.
                                     If None, returns the raw features from the backbone's pooler output or CLS token.
                                     Typically handled by the main CLIP model's projection head.
            tokenizer (IndicBERTTokenizer | None): The tokenizer instance. Used to check if token embeddings need resizing.
        """
        self.model_name = model_name
        self.pretrained = pretrained
        self.output_dim = output_dim
        self.tokenizer = tokenizer # Keep a reference if needed

        try:
            # Load model configuration
            config = AutoConfig.from_pretrained(model_name)
            self.feature_dim = config.hidden_size
            logger.info(f"Loading text model: {model_name} with pretrained={pretrained}")
            logger.info(f"Model hidden dimension: {self.feature_dim}")

            # Load the model
            if pretrained:
                self.backbone = AutoModel.from_pretrained(model_name, config=config)
            else:
                logger.warning(f"Initializing text model {model_name} from scratch (pretrained=False).")
                self.backbone = AutoModel.from_config(config)

            # Check if tokenizer requires resizing model embeddings
            if self.tokenizer is not None:
                current_embed_size = self.backbone.get_input_embeddings().weight.shape[0]
                if self.tokenizer.vocab_size != current_embed_size:
                    logger.warning(
                        f"Tokenizer vocab size ({self.tokenizer.vocab_size}) differs from model embedding size ({current_embed_size}). "
                        f"Resizing model token embeddings."
                    )
                    self.backbone.resize_token_embeddings(self.tokenizer.vocab_size)
                    # Update feature_dim based on potential new config after resize?
                    # Usually resize_token_embeddings only affects the embedding layer size,
                    # not the main hidden dim, but double-check if issues arise.
                    # self.feature_dim = self.backbone.config.hidden_size
                    logger.info(f"Model embedding size resized to {self.backbone.get_input_embeddings().weight.shape[0]}")
                else:
                    logger.info("Tokenizer vocab size matches model embedding size. No resizing needed.")


            # Optional projection layer (usually Identity here, handled in CLIP model)
            self.projection = nn.Identity()
            if output_dim is not None:
                self.projection = nn.Linear(self.feature_dim, output_dim)
                logger.info(f"Added text projection layer: Linear({self.feature_dim}, {output_dim})")

            # Placeholder for gradient checkpointing enabling
            self._gradient_checkpointing = False

        except Exception as e:
            logger.error(f"Failed to initialize TextEncoder with model {model_name}: {e}")
            raise

    def forward(self, input_ids: torch.Tensor, attention_mask: torch.Tensor) -> torch.Tensor:
        """
        Performs the forward pass to extract text features.

        Args:
            input_ids (torch.Tensor): Input token IDs (batch_size, sequence_length).
            attention_mask (torch.Tensor): Attention mask (batch_size, sequence_length).

        Returns:
            torch.Tensor: Output features (batch_size, feature_dim or output_dim).
                          Typically the embedding of the [CLS] token.
        """
        try:
            # Pass inputs through the backbone transformer model
            outputs = self.backbone(
                input_ids=input_ids,
                attention_mask=attention_mask,
                return_dict=True # Ensure outputs is a dict-like object
            )

            # Extract features - common strategies:
            # 1. Use pooler_output if available (often trained specifically for sentence tasks)
            # 2. Use the hidden state of the [CLS] token (first token)
            if hasattr(outputs, 'pooler_output') and outputs.pooler_output is not None:
                features = outputs.pooler_output
            elif hasattr(outputs, 'last_hidden_state'):
                # Use the [CLS] token embedding (index 0)
                features = outputs.last_hidden_state[:, 0]
            else:
                 logger.error("Could not extract features: Neither 'pooler_output' nor 'last_hidden_state' found in model outputs.")
                 # Attempt to return the raw output if desperate, but likely wrong shape
                 features = outputs[0][:, 0] # Highly speculative
                 # return torch.zeros((input_ids.shape[0], self.feature_dim), device=input_ids.device)

            # Apply optional projection
            projected_features = self.projection(features)
            return projected_features

        except Exception as e:
            logger.error(f"Error during TextEncoder forward pass: {e}")
            # Return dummy tensor or raise error
            return torch.zeros((input_ids.shape[0], self.output_dim or self.feature_dim), device=input_ids.device)

    def set_gradient_checkpointing(self, enable: bool = True):
        """Enables or disables gradient checkpointing for the backbone.
           Requires the Hugging Face model to support it.
        """
        if hasattr(self.backbone, 'gradient_checkpointing_enable') and hasattr(self.backbone, 'gradient_checkpointing_disable'):
            if enable:
                self.backbone.gradient_checkpointing_enable()
                logger.info("Enabled gradient checkpointing for text backbone.")
            else:
                self.backbone.gradient_checkpointing_disable()
                logger.info("Disabled gradient checkpointing for text backbone.")
            self._gradient_checkpointing = enable # Store state
        else:
            self._gradient_checkpointing = enable # Store desired state anyway
            logger.warning(f"Text backbone {self.model_name} might not directly support gradient checkpointing enable/disable methods.")

## Example Usage

In [ ]:
#| eval: false
if __name__ == '__main__':
    print("--- IndicBERT Example (No Projection) ---")
    try:
        # 1. Load the tokenizer first (assuming it's saved from 03_data_tokenization)
        # Use the path defined in core
        from indic_clip.core import TOKENIZER_PATH
        tokenizer = IndicBERTTokenizer.load_tokenizer(TOKENIZER_PATH, max_length=64)

        # 2. Instantiate the TextEncoder, passing the tokenizer
        text_encoder = TextEncoder(
            model_name=PRETRAINED_TOKENIZER_NAME, # Use the same base model as tokenizer
            pretrained=True,
            output_dim=None, # No projection in this example
            tokenizer=tokenizer # Pass the tokenizer instance
        )
        print(f"Encoder Type: {type(text_encoder)}")
        print(f"Feature Dimension: {text_encoder.feature_dim}")
        print(f"Output Dimension (after projection): {text_encoder.output_dim}")

        # Example gradient checkpointing control
        text_encoder.set_gradient_checkpointing(False)

        # 3. Create dummy tokenized input
        print("\nTokenizing example texts...")
        dummy_texts = [
            "गुलाबी पोशाक में बच्चा प्रवेश के रास्ते में सीढ़ियों के सेट पर चढ़ रहा है।",
            "कृष्णं वन्दे जगद्गुरुम्।",
            "A short English sentence."
        ]
        batch_encoding = tokenizer.tokenize(dummy_texts)
        input_ids = batch_encoding['input_ids']
        attention_mask = batch_encoding['attention_mask']

        print(f"Input IDs shape: {input_ids.shape}")
        print(f"Attention Mask shape: {attention_mask.shape}")

        # 4. Perform forward pass
        print("\nPerforming forward pass...")
        text_encoder.eval() # Set to evaluation mode
        with torch.no_grad():
            output_features = text_encoder(input_ids, attention_mask)

        print(f"Output features shape: {output_features.shape}")
        print(f"Output features dtype: {output_features.dtype}")

        # Assertions
        assert output_features.shape == (len(dummy_texts), text_encoder.feature_dim)
        assert output_features.dtype == torch.float32

        print("\nTextEncoder example executed successfully.")

    except FileNotFoundError as e:
        print(f"Error: {e}")
        print(f"Ensure the tokenizer has been saved to '{TOKENIZER_PATH}' by running 03_data_tokenization.ipynb first.")
    except Exception as e:
        print(f"An error occurred during TextEncoder example: {e}")
        import traceback
        traceback.print_exc()

--- IndicBERT Example (No Projection) ---


2025-04-21 14:27:51 - indic_clip.data.tokenization - INFO - Successfully loaded tokenizer: /workspace/indic-clip/models/tokenizer
2025-04-21 14:27:51 - indic_clip.data.tokenization - INFO - Custom special tokens already exist or none were specified.
2025-04-21 14:27:51 - indic_clip.data.tokenization - INFO - Tokenizer state loaded successfully from /workspace/indic-clip/models/tokenizer
2025-04-21 14:27:51 - __main__ - INFO - Loading text model: ai4bharat/indic-bert with pretrained=True
2025-04-21 14:27:51 - __main__ - INFO - Model hidden dimension: 768


pytorch_model.bin:   0%|          | 0.00/135M [00:00<?, ?B/s]

2025-04-21 14:27:58 - __main__ - WARNING - Tokenizer vocab size (200002) differs from model embedding size (200000). Resizing model token embeddings.
2025-04-21 14:27:58 - __main__ - INFO - Model embedding size resized to 200002
2025-04-21 14:27:58 - __main__ - INFO - Disabled gradient checkpointing for text backbone.


Encoder Type: <class '__main__.TextEncoder'>
Feature Dimension: 768
Output Dimension (after projection): None

Tokenizing example texts...
Input IDs shape: torch.Size([3, 64])
Attention Mask shape: torch.Size([3, 64])

Performing forward pass...
Output features shape: torch.Size([3, 768])
Output features dtype: torch.float32

TextEncoder example executed successfully.


model.safetensors:   0%|          | 0.00/135M [00:00<?, ?B/s]

In [ ]:
#| hide
import nbdev
nbdev.nbdev_export() # Run this in terminal to export